# Classification with Tabular Data using Scikit-Learn and Datalab


In this 5-minute quickstart tutorial, we use cleanlab with scikit-learn models to find potential label errors in a classification dataset with tabular (numeric/categorical) features. Tabular (or *structured*) data are typically organized in a row/column format and stored in a SQL database or file types like: CSV, Excel, or Parquet. Here we study the [German Credit](https://www.openml.org/d/31) dataset which contains 1,000 individuals described by 20 features, each labeled as either "good" or "bad" credit risk. cleanlab automatically shortlists _hundreds_ of examples from this dataset that confuse our ML model; many of which are potential label errors (due to annotator mistakes), edge cases, and otherwise ambiguous examples.

**Overview of what we'll do in this tutorial:**

- Build a simple credit risk classifier with scikit-learn's HistGradientBoostingClassifier.

- Use this classifier to compute out-of-sample predicted probabilities, `pred_probs`, via cross validation.

- Identify potential label errors in the data with cleanlab's `Datalab` class


<div class="alert alert-info">
Quickstart
<br/>
    
Already have (out-of-sample) `pred_probs` from a model trained on an existing set of labels? Run the code below to find any potential label errors in your dataset.

<div  class=markdown markdown="1" style="background:white;margin:16px">  
    
```ipython3 
from cleanlab import Datalab

lab = Datalab(data=your_dataset, label_name="column_name_of_labels")
lab.find_issues(pred_probs=your_pred_probs, issue_types={"label":{}})

lab.get_issues("label")
```
   
</div>
</div>

## 1. Install required dependencies


You can use `pip` to install all packages required for this tutorial as follows:

```ipython3
!pip install sklearn datasets
!pip install cleanlab[datalab]
# Make sure to install the version corresponding to this tutorial
# E.g. if viewing master branch documentation:
#     !pip install git+https://github.com/cleanlab/cleanlab.git
```

In [1]:
# Package installation (hidden on docs website).
dependencies = ["cleanlab", "sklearn", "datasets"]

if "google.colab" in str(get_ipython()):  # Check if it's running in Google Colab
    %pip install cleanlab  # for colab
    cmd = ' '.join([dep for dep in dependencies if dep != "cleanlab"])
    %pip install $cmd
else:
    missing_dependencies = []
    for dependency in dependencies:
        try:
            __import__(dependency)
        except ImportError:
            missing_dependencies.append(dependency)

    if len(missing_dependencies) > 0:
        print("Missing required dependencies:")
        print(*missing_dependencies, sep=", ")
        print("\nPlease install them before running the rest of this notebook.")

In [2]:
import random
import numpy as np

SEED = 100

np.random.seed(SEED)
random.seed(SEED)

## 2. Load and process the data


We first load the data features and labels (which are possibly noisy).


In [3]:
from sklearn.datasets import fetch_openml

credit_data = fetch_openml("credit-g", version=1)  # get the credit data from OpenML
X_raw = credit_data.data  # features (pandas DataFrame)
labels = credit_data.target  # labels (pandas Series)

Next we preprocess the data. Here we apply one-hot encoding to features with categorical data, and standardize features with numeric data. 

In [4]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

cat_features = X_raw.select_dtypes("category").columns
X_encoded = pd.get_dummies(X_raw, columns=cat_features, drop_first=True)

num_features = X_raw.select_dtypes("float64").columns
scaler = StandardScaler()
X_scaled = X_encoded.copy()
X_scaled[num_features] = scaler.fit_transform(X_encoded[num_features])

<div class="alert alert-info">
Bringing Your Own Data (BYOD)?

Assign your data's features to variable `X` and its labels to variable `labels` instead.

</div>

## 3. Select a classification model and compute out-of-sample predicted probabilities


Here we use a simple histogram-based gradient boosting model (similar to XGBoost), but you can choose any suitable scikit-learn model for this tutorial.


In [5]:
from sklearn.ensemble import HistGradientBoostingClassifier

clf = HistGradientBoostingClassifier()

To find potential labeling errors, cleanlab requires a probabilistic prediction from your model for every datapoint. However, these predictions will be _overfitted_ (and thus unreliable) for examples the model was previously trained on. cleanlab is intended to only be used with **out-of-sample** predicted probabilities, i.e., on examples held out from the model during the training.

K-fold cross-validation is a straightforward way to produce out-of-sample predicted probabilities for every datapoint in the dataset by training K copies of our model on different data subsets and using each copy to predict on the subset of data it did not see during training. An additional benefit of cross-validation is that it provides a more reliable evaluation of our model than a single training/validation split. We can obtain cross-validated out-of-sample predicted probabilities from any classifier via a simple scikit-learn wrapper:


In [6]:
from sklearn.model_selection import cross_val_predict

num_crossval_folds = 3  # for efficiency; values like 5 or 10 will generally work better
pred_probs = cross_val_predict(
    clf,
    X_scaled,
    labels,
    cv=num_crossval_folds,
    method="predict_proba",
)

## 4. Use datalab to find label issues


Based on the given labels and out-of-sample predicted probabilities, cleanlab can quickly help us identify poorly labeled instances in our data table. 

Here, we use `Datalab` to find potential label errors in our data. `Datalab` has several ways of loading the data. In this case, we’ll simply wrap the training features and noisy labels in a dictionary. We can instantiate our `Datalab` object with the dictionary created, and then pass in the model predicted probabilities we obtained above, and specify that we want to look for label errors by specifying that using the `issue_types` argument.

For a dataset with N examples from K classes, the labels should be a 1D array of length N and predicted probabilities should be a 2D (N x K) array.

In [7]:
from cleanlab import Datalab

data = {"X": X_scaled.values, "y": labels}

lab = Datalab(data, label_name="y")
lab.find_issues(pred_probs=pred_probs, issue_types={"label":{}})

Finding label issues ...
Audit complete. 198 issues found in the dataset.


From above, we see that `Datalab` has identified almost 200 label issues in the data. We can view the quality of all labels and details regarding if a label has been identified as an error using this `get_issues` method.

In [8]:
issue_results = lab.get_issues("label")
issue_results.head()

,is_label_issue,label_score,given_label,predicted_label
0,False,0.995546,good,good
1,False,0.948042,bad,bad
2,False,0.995657,good,good
3,True,0.433964,good,bad
4,False,0.581302,bad,bad


To view the most severe label issues, we can sort the dataframe above by the `label_score` column (a lower score represents that the label is more likely to be erroneous). 

Let's review some of the most likely label errors:

In [9]:
sorted_issues = issue_results.sort_values("label_score").index
X_raw.iloc[sorted_issues].assign(label=labels.iloc[sorted_issues]).head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,label
949,no checking,24.0,existing paid,radio/tv,3621.0,100<=X<500,>=7,2.0,male single,none,...,car,31.0,none,own,2.0,skilled,1.0,none,yes,bad
505,no checking,10.0,existing paid,new car,1309.0,no known savings,1<=X<4,4.0,male single,guarantor,...,life insurance,27.0,none,own,1.0,unskilled resident,1.0,none,yes,bad
963,no checking,24.0,existing paid,radio/tv,2397.0,500<=X<1000,>=7,3.0,male single,none,...,car,35.0,bank,own,2.0,skilled,1.0,yes,yes,bad
978,no checking,24.0,delayed previously,new car,2538.0,<100,>=7,4.0,male single,none,...,car,47.0,none,own,2.0,unskilled resident,2.0,none,yes,bad
228,no checking,9.0,existing paid,radio/tv,1478.0,<100,4<=X<7,4.0,male single,none,...,car,22.0,none,own,1.0,skilled,1.0,none,yes,bad


These examples appear the most suspicious to our model and should be carefully re-examined. Perhaps the original annotators missed something when deciding on the labels for these individuals. This is a straightforward approach to visualize the rows in a data table that might be mislabeled.
